In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

%matplotlib inline

In [3]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

In [222]:
#function to strip timezone from datetime
def strip_tz(row):
    return row['localhour'].strftime('%Y-%m-%d %H:%M:%S')

In [14]:
pd.set_option('display.max_columns', None) 

In [121]:
df_meta_physical = pickle.load(open('pickle_files/df_meta_physical.p','rb'))

In [122]:
second_floors = df_meta_physical[df_meta_physical.second_floor_square_footage > 0]
third_floors = df_meta_physical[df_meta_physical.third_floor_square_footage > 0]

In [123]:
#add number of floors as feature
df_meta_physical.loc[:, 'Floors'] = 1
df_meta_physical.loc[second_floors.index, 'Floors'] = 2
df_meta_physical.loc[third_floors.index, 'Floors'] = 3

In [124]:
df_regression = df_meta_physical.loc[:,['dataid','building_type_x','pv_x','house_construction_year_x', 'total_square_footage_x', 'Floors']]

In [125]:
df_regression.index = df_regression['dataid']

In [126]:
df_regression.loc[4874,:]

dataid                                     4874
building_type_x              Single-Family Home
pv_x                                        yes
house_construction_year_x                  2010
total_square_footage_x                     2021
Floors                                        2
Name: 4874, dtype: object

Add energy usage to each home- decide if we want to do month...

In [111]:
from datetime import datetime, timedelta

In [107]:
df_meta_refined = pickle.load(open('pickle_files/df_meta_refined.p', 'rb'))

In [108]:
egauge_8872 = df_meta_refined.loc[8872,'egauge_max_time']

In [110]:
end_8872 = datetime.strptime('{}-{}-{} 00:00:00'.format(egauge_8872.year, egauge_8872.month, egauge_8872.day), '%Y-%m-%d %H:%M:%S')

In [114]:
time_change = timedelta(days=-1)

In [115]:
end_8872, end_8872+time_change

(datetime.datetime(2013, 6, 10, 0, 0), datetime.datetime(2013, 6, 9, 0, 0))

In [236]:
#function to find latest month of egauge data
def find_egauge_dates(dataid, final_date=None):
    
    df = pickle.load(open('pickle_files/df_{}.p'.format(dataid),'rb'))
    
    if not final_date:
        final_date = df_meta_refined.loc[dataid, 'egauge_max_time']
    
    final_date = datetime.strftime(final_date,'%Y-%m-%d %H:%M:%S')
    final_date = datetime.strptime(final_date, '%Y-%m-%d %H:%M:%S')
        
    max_date = datetime.strptime('2016-10-31 00:00:00', '%Y-%m-%d %H:%M:%S')
    
    if final_date > max_date:
        final_date = max_date
    
    end_date = datetime.strptime('{}-{}-{} 00:00:00'.format(final_date.year, final_date.month, final_date.day), '%Y-%m-%d %H:%M:%S')
    time_delta = timedelta(days=30)
    start_date = end_date - time_delta
    
    return start_date, end_date

In [203]:
start_8047, end_8047 = find_egauge_dates(8047)

End Date:2014-04-07 00:00:00
Start Date:2014-03-08 00:00:00


In [192]:
df_test_1464 = pickle.load(open('pickle_files/df_1464.p','rb'))
df_test_1464.index = df_test_1464['localhour']

In [237]:
#function to calculate total energy usage for time period
def calculate_energy_usage(dataid, final_date=None):
    
    start, end = find_egauge_dates(dataid, final_date)
    df = pickle.load(open('pickle_files/df_{}.p'.format(dataid), 'rb'))
    
    df.index = df.apply(strip_tz, axis=1)
    
    energy_usage = sum(df.loc[str(start):str(end), 'use'])
#     energy_usage = sum(df.loc[start:end, 'use'])
    
    return energy_usage

In [233]:
df_8047 = pickle.load(open('pickle_files/df_8047.p', 'rb'))
df_8047.index = df_8047['localhour']

In [234]:
calculate_energy_usage(8047)

End Date:2014-04-07 00:00:00
Start Date:2014-03-08 00:00:00


263.33581666666623

In [240]:
#add energy usage for all buildings to df_regression
for i in df_regression.index:
    try:
        energy_usage = calculate_energy_usage(i)
        df_regression.loc[i,'Energy_Usage'] = energy_usage
    except (FileNotFoundError, TypeError) as e :
        print(e)

[Errno 2] No such file or directory: 'pickle_files/df_8872.p'
unsupported operand type(s) for +: 'int' and 'NoneType'
unsupported operand type(s) for +: 'int' and 'NoneType'
unsupported operand type(s) for +: 'int' and 'NoneType'
unsupported operand type(s) for +: 'int' and 'NoneType'


In [249]:
#remove buildings with insufficient data
df_regression = df_regression[df_regression.Energy_Usage > 0]

In [250]:
df_regression.loc[:,'Building_Age'] = 2017 - df_regression.loc[:,'house_construction_year_x']

In [251]:
df_meta_physical

,dataid,active_record,building_type_x,program_579,program_baseline,program_energy_internet_demo,program_lg_appliance,program_verizon,program_ccet_group,program_civita_group,city,state,pv_x,date_enrolled,date_withdrawn,house_construction_year_x,total_square_footage_x,first_floor_square_footage,second_floor_square_footage,third_floor_square_footage,half_floor_square_footage,lower_level_square_footage,audit_2011,audit_2013_2014,survey_2011,survey_2012,survey_2013,survey_2014,number_of_nests,indoor_temp_min_time,indoor_temp_max_time,gas_ert_min_time,gas_ert_max_time,water_ert_min_time,water_ert_max_time,egauge_min_time,egauge_max_time,use,grid,gen,air1,air2,air3,airwindowunit1,aquarium1,bathroom1,bathroom2,bedroom1,bedroom2,bedroom3,bedroom4,bedroom5,car1,clotheswasher1,clotheswasher_dryg1,diningroom1,diningroom2,dishwasher1,disposal1,drye1,dryg1,freezer1,furnace1,furnace2,garage1,garage2,heater1,housefan1,icemaker1,jacuzzi1,kitchen1,kitchen2,kitchenapp1,kitchenapp2,lights_plugs1,lights_plugs2,lights_plugs3,lights_plugs4,lights_plugs5,lights_plugs6,livingroom1,livingroom2,microwave1,office1,outsidelights_plugs1,outsidelights_plugs2,oven1,oven2,pool1,pool2,poollight1,poolpump1,pump1,range1,refrigerator1,refrigerator2,security1,shed1,sprinkler1,utilityroom1,venthood1,waterheater1,waterheater2,winecooler1,building_type_y,pv_y,house_construction_year_y,total_square_footage_y,Floors
0,8872,None,Single-Family Home,None,yes,yes,None,None,CCET - Control,None,Austin,Texas,yes,2011-02-01,2013-06-10,2008.0,1422.0,1422.0,0.0,0.0,NaN,NaN,yes,,yes,yes,,,NaN,None,None,None,None,None,None,2012-05-02 01:00:00-05:00,2013-06-10 00:59:00-05:00,yes,yes,yes,yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Single-Family Home,yes,2008.0,1422.0,1
1,1464,None,Single-Family Home,None,yes,yes,None,None,CCET - Control,None,Austin,Texas,yes,2011-02-01,2014-03-06,2007.0,2345.0,1385.0,960.0,0.0,NaN,NaN,yes,,yes,yes,,,NaN,None,None,2012-04-13 09:26:26-05:00,2014-02-06 23:53:33-06:00,2012-05-01 13:00:00-05:00,2014-02-06 23:58:03-06:00,2012-10-19 19:00:00-05:00,2014-03-05 17:59:00-06:00,yes,yes,yes,yes,None,None,None,None,None,None,yes,yes,yes,None,None,None,yes,None,None,None,yes,yes,None,yes,None,None,None,None,None,None,None,None,None,None,None,yes,None,yes,None,None,None,None,None,None,None,yes,None,None,None,yes,None,None,None,None,None,None,yes,yes,None,yes,None,yes,None,None,None,None,None,Single-Family Home,yes,2007.0,2345.0,2
2,8047,None,Single-Family Home,None,yes,yes,None,None,CCET - Pricing Trial,None,Austin,Texas,yes,2011-02-01,2014-04-07,2009.0,3245.0,1779.0,1466.0,0.0,NaN,NaN,yes,,yes,,yes,,NaN,None,None,2012-04-11 18:59:52-05:00,2013-05-10 16:26:10-05:00,None,None,2013-04-16 00:00:00-05:00,2014-04-07 00:00:00-05:00,yes,yes,yes,yes,None,None,None,None,yes,None,None,None,None,None,None,None,yes,None,None,None,yes,yes,None,yes,yes,yes,None,None,None,None,None,None,None,None,None,yes,yes,None,None,None,None,None,None,None,None,yes,None,None,None,yes,None,None,None,None,None,None,None,yes,None,None,None,None,None,yes,None,None,None,Single-Family Home,yes,2009.0,3245.0,2
3,8597,None,Single-Family Home,None,yes,yes,None,None,CCET - UT Text,None,Austin,Texas,yes,2011-02-01,2014-10-31,2008.0,2122.0,1162.0,960.0,0.0,NaN,NaN,yes,,yes,,,,NaN,None,None,2012-04-11 18:59:53-05:00,2013-05-10 16:28:22-05:00,None,None,2012-10-26 19:00:00-05:00,2014-10-21 18:59:00-05:00,yes,yes,yes,yes,None,None,None,None,None,None,yes,None,None,None,None,None,yes,None,yes,None,yes,yes,yes,None,None,yes,None,None,None,None,None,None,None,None,None,yes,yes,yes,yes,None,None,None,None,None,None,None,None,None,None,yes,None,None,None,None,None,None,yes,yes,None,None,None,None,None,None,None,None,None,Single-Family Home,yes,2